In [1]:
!pip install duckdb pandas pytz google-cloud-bigquery --quiet

zsh:1: command not found: pip


In [2]:
import json
import sys
import duckdb
import pytz
import pandas as pd
from pathlib import Path
from datetime import datetime
from google.cloud import bigquery
from concurrent.futures import ThreadPoolExecutor, as_completed

In [4]:
!/opt/homebrew/bin/gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=3QXvNct9llllw6O7SflZmjMGoURgdg&access_type=offline&code_challenge=UMW5kw4jT4Gc2KV6xAKgWdNAwWYPmfBmzdhnpHLVt1E&code_challenge_method=S256


Credentials saved to file: [/Users/machadomath/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "site-hypr" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [5]:
bq_client = bigquery.Client(project='site-hypr')

In [6]:
duckdb_conn = duckdb.connect(database=':memory:')

In [7]:
base_dirs = [
    '/Users/machadomath/Documents/ATUALIZAÇÃO RMNF/INFOSIMPES LOTES/NF_INFOSIMPLES_COBASI_OUT-8K_PENDENTE.zip-responses-20251029-1-say8qg',
    '/Users/machadomath/Documents/ATUALIZAÇÃO RMNF/INFOSIMPES LOTES/NF_INFOSIMPLES_OBOTICARIO_OUT-16K_PENDENTE-responses-20251029-1-71d3ja'
]

In [8]:
all_json_files = [
    str(p) for base in base_dirs for p in Path(base).glob("*.json")
]

In [10]:
all_json_files

['/Users/machadomath/Documents/ATUALIZAÇÃO RMNF/INFOSIMPES LOTES/NF_INFOSIMPLES_COBASI_OUT-8K_PENDENTE.zip-responses-20251029-1-say8qg/chave-35250653153938001775650030000036181317916830--1787.json',
 '/Users/machadomath/Documents/ATUALIZAÇÃO RMNF/INFOSIMPES LOTES/NF_INFOSIMPLES_COBASI_OUT-8K_PENDENTE.zip-responses-20251029-1-say8qg/chave-35250553153938002313590005017724400266967232--4082.json',
 '/Users/machadomath/Documents/ATUALIZAÇÃO RMNF/INFOSIMPES LOTES/NF_INFOSIMPLES_COBASI_OUT-8K_PENDENTE.zip-responses-20251029-1-say8qg/chave-35250253153938009164590007403792362973294306--5892.json',
 '/Users/machadomath/Documents/ATUALIZAÇÃO RMNF/INFOSIMPES LOTES/NF_INFOSIMPLES_COBASI_OUT-8K_PENDENTE.zip-responses-20251029-1-say8qg/chave-35250353153938001775590006408876962792015640--7320.json',
 '/Users/machadomath/Documents/ATUALIZAÇÃO RMNF/INFOSIMPES LOTES/NF_INFOSIMPLES_COBASI_OUT-8K_PENDENTE.zip-responses-20251029-1-say8qg/chave-35250353153938001775590006408876895539671291--8379.js

In [ ]:
dfs = []
for file in all_json_files:
    tmp = pd.read_json(file, lines=True)
    tmp['token'] = tmp['header'].apply(
        lambda x: x['parameters'].get('chave') or x['parameters'].get('nfce') 
        if isinstance(x, dict) and 'parameters' in x and isinstance(x['parameters'], dict) 
        else None
    )
    tmp = tmp[tmp['token'].notnull()]  # Remove rows with null tokens
    dfs.append(tmp)

In [13]:
agg_df = pd.concat(dfs, ignore_index=True)
agg_df['full_json'] = agg_df.apply(lambda row: json.dumps(row.to_dict(), ensure_ascii=False), axis=1)
agg_df = agg_df[['token', 'full_json']]
agg_df.columns = ['token', 'data']
agg_df.head()


,token,data
0,35250653153938001775650030000036181317916830,"{""code"": 608, ""code_message"": ""Os parâmetros f..."
1,35250553153938002313590005017724400266967232,"{""code"": 200, ""code_message"": ""A requisição fo..."
2,35250253153938009164590007403792362973294306,"{""code"": 200, ""code_message"": ""A requisição fo..."
3,None,"{""code"": 603, ""code_message"": ""O token informa..."
4,None,"{""code"": 603, ""code_message"": ""O token informa..."


In [42]:
tmp = agg_df.copy()
total = tmp["token"].nunique()
tmp["code"] =tmp["data"].apply(lambda x: json.loads(x)["code"])
tmp = tmp.groupby("code")["token"].nunique().reset_index(name="distinct_token_count")
tmp["percent"] = (tmp["distinct_token_count"] / total * 100).round(2)
tmp

,code,distinct_token_count,percent
0,200,24996,99.98
1,608,2,0.01
2,612,2,0.01


In [43]:
agg_df = agg_df.copy()

rows_to_insert = []

for row in agg_df.itertuples():
    try:
        rows_to_insert.append({
            "token": row.token,
            "moment": datetime.now(pytz.timezone('America/Sao_Paulo')).isoformat(),
            "data": row.data,
            "status": 'PROCESSADO'
        })
    except Exception as e:
        print(f"Error for token {row.token}: {e}")


In [44]:
BATCH_SIZE = 1000
for i in range(0, len(rows_to_insert), BATCH_SIZE):
    batch = rows_to_insert[i:i+BATCH_SIZE]
    errors = bq_client.insert_rows_json('site-hypr.staging.fireho_processed_tokens', batch)
    if errors:
        print(f"Errors in batch {i//BATCH_SIZE}: {errors}")